## Local Alignment

In [6]:
import numpy

def LocalAlign(v, w, scorematrix, indel):
    s = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    b = numpy.zeros((len(v)+1,len(w)+1), dtype="int32")
    for i in range(1,len(v)+1):
        for j in range(1,len(w)+1):
            if (j == 0):
                if (i > 0):
                    s[i,j] = max(s[i-1,j] + indel, 0)
                    b[i,j] = 1
                continue
            if (i == 0):
                s[i,j] = max(s[i,j-1] + indel, 0)
                b[i,j] = 2
                continue
            score = s[i-1,j-1] + scorematrix[v[i-1],w[j-1]]
            vskip = s[i-1,j] + indel
            wskip = s[i,j-1] + indel
            s[i,j] = max(vskip, wskip, score, 0)
            if (s[i,j] == vskip):
                b[i,j] = 1
            elif (s[i,j] == wskip):
                b[i,j] = 2
            elif (s[i,j] == score):
                b[i,j] = 3
            else:
                b[i,j] = 0
    return (s, b)

match = {('A','A'):  5, ('A','C'): -4, ('A','G'): -4, ('A','T'): -4,
         ('C','A'): -4, ('C','C'):  5, ('C','G'): -4, ('C','T'): -4,
         ('G','A'): -4, ('G','C'): -4, ('G','G'):  5, ('G','T'): -4,
         ('T','A'): -4, ('T','C'): -4, ('T','G'): -4, ('T','T'):  5}

w = "GCTGGAAGGCAT"
v = "GCAGAGCACT"

s, b = LocalAlign(v,w,match,-7)
print(s)
print()
print(b)

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  5  0  0  5  5  0  0  5  5  0  0  0]
 [ 0  0 10  3  0  1  1  0  0  1 10  3  0]
 [ 0  0  3  6  0  0  6  6  0  0  3 15  8]
 [ 0  5  0  0 11  5  0  2 11  5  0  8 11]
 [ 0  0  1  0  4  7 10  5  4  7  1  5  4]
 [ 0  5  0  0  5  9  3  6 10  9  3  0  1]
 [ 0  0 10  3  0  2  5  0  3  6 14  7  0]
 [ 0  0  3  6  0  0  7 10  3  0  7 19 12]
 [ 0  0  5  0  2  0  0  3  6  0  5 12 15]
 [ 0  0  0 10  3  0  0  0  0  2  0  5 17]]

[[0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 3 3 0 0 3 3 0 0 0]
 [0 0 3 2 0 3 3 0 0 3 3 2 0]
 [0 0 1 3 0 0 3 3 0 0 1 3 2]
 [0 3 0 0 3 3 0 3 3 3 0 1 3]
 [0 0 3 0 1 3 3 3 1 3 3 3 1]
 [0 3 0 0 3 3 1 3 3 3 3 0 3]
 [0 0 3 2 0 1 3 0 1 3 3 2 2]
 [0 0 1 3 0 0 3 3 2 0 1 3 2]
 [0 0 3 0 3 0 1 1 3 0 3 1 3]
 [0 0 0 3 2 0 0 0 0 3 0 1 3]]


## Local Alignment
The difference here is subtle. Backtracking ends when it reaches a direction of '0' (a free ride) is reached rather than when (i,j) = (0,0)

In [8]:
def Alignment(b,v,w,i,j):
    if ((i == 0) and (j == 0)):
        return ['','']
    if (b[i,j] == 3):
        result = Alignment(b,v,w,i-1,j-1)
        result[0] += v[i-1]
        result[1] += w[j-1]
        return result
    if (b[i,j] == 2):
        result = Alignment(b,v,w,i,j-1)
        result[0] += "_"
        result[1] += w[j-1]
        return result
    if (b[i,j] == 1):
        result = Alignment(b,v,w,i-1,j)
        result[0] += v[i-1]
        result[1] += "_"
        return result

In [10]:
def LocalAlignment(b,v,w,i,j):
    if (b[i,j] == 0):
        return ['','']
    if (b[i,j] == 3):
        result = LocalAlignment(b,v,w,i-1,j-1)
        result[0] += v[i-1]
        result[1] += w[j-1]
        return result
    if (b[i,j] == 2):
        result = LocalAlignment(b,v,w,i,j-1)
        result[0] += "_"
        result[1] += w[j-1]
        return result
    if (b[i,j] == 1):
        result = LocalAlignment(b,v,w,i-1,j)
        result[0] += v[i-1]
        result[1] += "_"
        return result
    
maxij = numpy.unravel_index(s.argmax(), s.shape)
print(maxij, s[maxij])
vsub, wsub = LocalAlignment(b,v,w,maxij[0],maxij[1])
print(v)
print(w)
print(vsub)
print(wsub)

(8, 11) 19
GCAGAGCACT
GCTGGAAGGCAT
GCAGAGCA
GAAG_GCA
